In [10]:
import pandas as pd

df = pd.read_csv("All_Merged.csv")
df

,Event Time,Location Tag,Source,Condition,Action,Priority,Description,Value,Units
0,2025-01-01 00:00:00,SERVER_PVC,DAILY RP,PERIODIC,NaN,J 00,Report Periodic Request,12,NaN
1,2025-01-01 00:00:03,1600,PI1601,PVLOLO,NaN,J 00,HE1601 OUTLET,0.239591,kg/cm2G
2,2025-01-01 00:00:04,1500,TI1507,PVLOW,ACK,NaN,TW1501 BOTTOM,NaN,NaN
3,2025-01-01 00:00:05,IOLINK04_1,AIOR-704,OP Fail in circuit/field wire,OK,H 15,NaN,Channel_18_B,NaN
4,2025-01-01 00:00:07,1200,TIC1203,PVHIHI,NaN,H 00,HE1203 D/S TEMP,88.7875,C
...,...,...,...,...,...,...,...,...,...
903349,2025-03-17 10:55:00,1300,AI1301,PVLOW,NaN,L 00,VE1306 INLET,5.99933,ph
903350,2025-03-17 10:55:00,1300,AI1301,PVLOW,ACK,NaN,VE1306 INLET,NaN,NaN
903351,2025-03-17 10:55:00,SERVER_PVC,NaN,Demand,NaN,NaN,CSV export in progress...,NaN,NaN
903352,2025-03-17 10:56:00,SERVER_PVC,NaN,Demand,NaN,NaN,CSV export in progress...,NaN,NaN


# Logics


- An Alarm is in active state till no action perform on it and it is considered as 1 alarm till no action performing on it 
- If no one perform any action on an single source alarm for long duration then it will be considered as stale alarm
- And if an alarm from same source occurs repeatedly in short duration with the actions performed on it by operator as well
then it should be considered as Chattering

- with max alarm count in a flood a source could be considered as bad actor
- unhealthy source if an alarm occurs >= 10 in 10 minutes duration
- If multiple sources are in active state or multiple alarms occurs from multiple sources in 10 min window or more then it would be considered as Flood


In [4]:
# import pandas as pd

# # =====================================================
# # === KPI ENGINE FOR ALARM MANAGEMENT SYSTEM BACKEND ===
# # =====================================================

# def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
#     """Prepare and clean alarm event data"""
#     df = df.copy()
#     df['Event Time'] = pd.to_datetime(df['Event Time'], errors='coerce')
#     df['Action'] = df['Action'].astype(str).str.strip().replace({'nan': ''})
#     df = df.dropna(subset=['Event Time', 'Source'])
#     df = df.sort_values('Event Time')
#     df['Action'] = df['Action'].fillna('').str.upper().str.strip()
#     return df


# def get_alarm_cycles(df: pd.DataFrame) -> pd.DataFrame:
#     """Extract complete/incomplete alarm cycles for each source"""
#     alarm_cycles = []

#     for source, group in df.groupby('Source'):
#         group = group.sort_values('Event Time').reset_index(drop=True)
#         current_alarm = None

#         for _, row in group.iterrows():
#             action = row['Action']
#             time = row['Event Time']

#             if action == '' and current_alarm is None:
#                 current_alarm = {'source': source, 'start_time': time, 'ack_time': None, 'ok_time': None}

#             elif action == 'ACK' and current_alarm and current_alarm['ack_time'] is None:
#                 current_alarm['ack_time'] = time

#             elif action == 'OK' and current_alarm:
#                 current_alarm['ok_time'] = time
#                 alarm_cycles.append(current_alarm)
#                 current_alarm = None  # Reset after completion

#         # Handle unclosed alarm
#         if current_alarm:
#             alarm_cycles.append(current_alarm)

#     cycles_df = pd.DataFrame(alarm_cycles)
#     if not cycles_df.empty:
#         cycles_df['ack_delay'] = (cycles_df['ack_time'] - cycles_df['start_time']).dt.total_seconds() / 60
#         cycles_df['ok_delay'] = (cycles_df['ok_time'] - cycles_df['start_time']).dt.total_seconds() / 60
#     return cycles_df


# # =====================================================
# # ============== KPI CALCULATION FUNCTIONS ============
# # =====================================================

# def kpi_total_alarms(cycles_df: pd.DataFrame) -> int:
#     return len(cycles_df)


# def kpi_unique_sources(df: pd.DataFrame) -> int:
#     return df['Source'].nunique()


# def kpi_avg_ack_time(cycles_df: pd.DataFrame) -> float:
#     return round(cycles_df['ack_delay'].mean(), 2) if not cycles_df.empty else 0


# def kpi_avg_ok_time(cycles_df: pd.DataFrame) -> float:
#     return round(cycles_df['ok_delay'].mean(), 2) if not cycles_df.empty else 0


# def kpi_stale_alarms(cycles_df: pd.DataFrame, threshold_min: int = 30) -> int:
#     """Stale if no OK for > threshold"""
#     if cycles_df.empty:
#         return 0
#     stale = cycles_df[
#         (cycles_df['ok_time'].isna()) |
#         ((cycles_df['ok_time'] - cycles_df['start_time']).dt.total_seconds() / 60 > threshold_min)
#     ]
#     return len(stale)


# def kpi_chattering_alarms(cycles_df: pd.DataFrame, gap_min: int = 10) -> int:
#     """Count chattering alarms per source based on time gap between alarms"""
#     if cycles_df.empty:
#         return 0
#     chatter_count = 0
#     for source, group in cycles_df.groupby('source'):
#         group = group.sort_values('start_time').reset_index(drop=True)
#         group['gap'] = group['start_time'].diff().dt.total_seconds() / 60
#         chatter_count += group[group['gap'] < gap_min].shape[0]
#     return chatter_count


# # =====================================================
# # =============== MAIN KPI WRAPPER ====================
# # =====================================================

# def compute_all_kpis(df: pd.DataFrame) -> dict:
#     """Main wrapper to compute all KPIs efficiently"""
#     df = preprocess_data(df)
#     cycles_df = get_alarm_cycles(df)

#     total_alarms = kpi_total_alarms(cycles_df)
#     stale_alarms = kpi_stale_alarms(cycles_df)
#     chatter_alarms = kpi_chattering_alarms(cycles_df)

#     return {
#         "total_alarms": total_alarms,
#         "unique_sources": kpi_unique_sources(df),
#         "average_ack_time_min": kpi_avg_ack_time(cycles_df),
#         "average_ok_time_min": kpi_avg_ok_time(cycles_df),
#         "stale_alarms": stale_alarms,
#         "stale_alarm_percent": round((stale_alarms / total_alarms * 100), 2) if total_alarms else 0,
#         "chattering_alarms": chatter_alarms,
#         "chattering_alarm_percent": round((chatter_alarms / total_alarms * 100), 2) if total_alarms else 0
#     }


In [15]:
# kpis = compute_all_kpis(df)
# kpis

{'total_alarms': 81579,
 'unique_sources': 1230,
 'average_ack_time_min': np.float64(165.49),
 'average_ok_time_min': np.float64(197.93),
 'stale_alarms': 28472,
 'stale_alarm_percent': 34.9,
 'chattering_alarms': 32007,
 'chattering_alarm_percent': 39.23}

In [25]:
# alarm_kpi_service.py
import pandas as pd

# thresholds
STALE_THRESHOLD_MIN = 60
CHATTER_THRESHOLD_MIN = 10

# ---------- CORE KPI LOGIC  ----------
def count_unique_alarms(group):
    alarm_count = 0
    state = "IDLE"
    for _, row in group.iterrows():
        action = str(row["Action"]).upper().strip()
        if action == "":
            if state == "IDLE" or state == "ACKED":
                alarm_count += 1
                state = "ACTIVE"
        elif action == "ACK" and state == "ACTIVE":
            state = "ACKED"
        elif action == "OK":
            state = "IDLE"
    return alarm_count


def detect_stale_chatter(group):
    group = group.sort_values("Event Time")
    stale_flags = []
    chatter_flags = []

    last_alarm_time = None
    recent_alarms = []

    for i, row in group.iterrows():
        t = row["Event Time"]
        action = row["Action"]

        # Detect stale alarms
        if action == "ALARM":
            if last_alarm_time is None:
                stale_flags.append(False)
            else:
                stale_flags.append((t - last_alarm_time).total_seconds() / 60 > STALE_THRESHOLD_MIN)
            last_alarm_time = t
        elif action == "ACK":
            stale_flags.append(False)
        else:
            stale_flags.append(False)

        # Detect chattering (too many alarms in a short time)
        recent_alarms = [x for x in recent_alarms if (t - x).total_seconds() / 60 <= CHATTER_THRESHOLD_MIN]
        recent_alarms.append(t)
        chatter_flags.append(len(recent_alarms) > CHATTER_THRESHOLD_MIN)

    group["is_stale"] = stale_flags
    group["is_chattering"] = chatter_flags
    return group



def get_alarm_cycles(df):
    cycles = []
    for src, g in df.groupby("Source"):
        g = g.sort_values("Event Time").reset_index(drop=True)
        cur = None
        for _, r in g.iterrows():
            a = str(r["Action"]).upper().strip()
            t = r["Event Time"]
            if a == "" and cur is None:
                cur = {"source": src, "start_time": t, "ack_time": None, "ok_time": None}
            elif a == "ACK" and cur and cur["ack_time"] is None:
                cur["ack_time"] = t
            elif a == "OK" and cur:
                cur["ok_time"] = t; cycles.append(cur); cur = None
        if cur: cycles.append(cur)
    dfc = pd.DataFrame(cycles)
    if not dfc.empty:
        dfc["ack_delay"] = (dfc["ack_time"] - dfc["start_time"]).dt.total_seconds() / 60
        dfc["ok_delay"] = (dfc["ok_time"] - dfc["start_time"]).dt.total_seconds() / 60
    return dfc


def calculate_alarm_kpis(df):
    df = df.copy()
    df["Event Time"] = pd.to_datetime(df["Event Time"])
    df = df.sort_values(["Source", "Event Time"])

    """Prepare and clean alarm event data"""
    df = df.copy()
    df['Event Time'] = pd.to_datetime(df['Event Time'], errors='coerce')
    df['Action'] = df['Action'].astype(str).str.strip().replace({'nan': ''})
    # df = df.dropna(subset=['Event Time', 'Source'])
    # df = df.sort_values('Event Time')
    df = df.sort_values(["Source", "Event Time"])
    # df['Action'] = df['Action'].fillna('').str.upper().str.strip()

    unique = df.groupby("Source").apply(count_unique_alarms).reset_index(name="Unique_Alarms")
    analyzed = df.groupby("Source", group_keys=False).apply(detect_stale_chatter)
    stale = analyzed.groupby("Source")["is_stale"].sum().reset_index(name="Stale_Count")
    chat = analyzed.groupby("Source")["is_chattering"].sum().reset_index(name="Chattering_Count")

    cycles = get_alarm_cycles(df)
    if cycles.empty:
        avg_ack = avg_ok = completion = 0
    else:
        avg_ack = cycles["ack_delay"].mean()
        avg_ok = cycles["ok_delay"].mean()
        completion = cycles["ok_time"].notnull().mean() * 100

    df["Date"] = df["Event Time"].dt.date
    per_day = df.groupby("Date").size()
    hrs = (df["Event Time"].max() - df["Event Time"].min()).total_seconds() / 3600
    avg_day = per_day.mean()
    avg_hr = len(df) / hrs if hrs else 0
    avg_10m = avg_hr / 6
    overlimit_pct = (per_day > 288).mean() * 100

    summary = unique.merge(stale, on="Source", how="outer").merge(chat, on="Source", how="outer").fillna(0)
    kpis = dict(
        avg_ack_delay_min=avg_ack,
        avg_ok_delay_min=avg_ok,
        completion_rate_pct=completion,
        avg_alarms_per_day=avg_day,
        avg_alarms_per_hour=avg_hr,
        avg_alarms_per_10min=avg_10m,
        days_over_288_alarms_pct=overlimit_pct,
    )
    return summary, kpis, cycles


In [26]:
summary, kpis, cycles = calculate_alarm_kpis(df)

    # Prepare JSON-safe payloads
    
print("global_kpis", kpis)
print("per_source_summary", summary.to_dict(orient="records"))
print("alarm_cycles", cycles.to_dict(orient="records"))
    

C:\Users\hmaba\AppData\Local\Temp\ipykernel_22128\3681710263.py:96: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique = df.groupby("Source").apply(count_unique_alarms).reset_index(name="Unique_Alarms")
C:\Users\hmaba\AppData\Local\Temp\ipykernel_22128\3681710263.py:97: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  analyzed = df.groupby("Source", group_keys=False).apply(detect_stale_chatter)


global_kpis {'avg_ack_delay_min': np.float64(140.80233469349903), 'avg_ok_delay_min': np.float64(185.6824968371187), 'completion_rate_pct': np.float64(98.88366627497062), 'avg_alarms_per_day': np.float64(12044.72), 'avg_alarms_per_hour': 498.833382417906, 'avg_alarms_per_10min': 83.13889706965101, 'days_over_288_alarms_pct': np.float64(100.0)}
per_source_summary [{'Source': '$ACTIVITY_330413F2352AB609', 'Unique_Alarms': 1, 'Stale_Count': 0, 'Chattering_Count': 44375}, {'Source': 'AAH10023', 'Unique_Alarms': 1, 'Stale_Count': 0, 'Chattering_Count': 0}, {'Source': 'AAH10026', 'Unique_Alarms': 2, 'Stale_Count': 0, 'Chattering_Count': 0}, {'Source': 'AAH10027', 'Unique_Alarms': 22, 'Stale_Count': 0, 'Chattering_Count': 2}, {'Source': 'AAH10028', 'Unique_Alarms': 1, 'Stale_Count': 0, 'Chattering_Count': 0}, {'Source': 'AERATION', 'Unique_Alarms': 1, 'Stale_Count': 0, 'Chattering_Count': 0}, {'Source': 'AERATION_CM', 'Unique_Alarms': 4, 'Stale_Count': 0, 'Chattering_Count': 0}, {'Source': 'A

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
kpis

{'avg_ack_delay_min': np.float64(140.80233469349903),
 'avg_ok_delay_min': np.float64(185.6824968371187),
 'completion_rate_pct': np.float64(98.88366627497062),
 'avg_alarms_per_day': np.float64(12044.72),
 'avg_alarms_per_hour': 498.833382417906,
 'avg_alarms_per_10min': 83.13889706965101,
 'days_over_288_alarms_pct': np.float64(100.0)}

In [28]:
summary

,Source,Unique_Alarms,Stale_Count,Chattering_Count
0,$ACTIVITY_330413F2352AB609,1,0,44375
1,AAH10023,1,0,0
2,AAH10026,2,0,0
3,AAH10027,22,0,2
4,AAH10028,1,0,0
...,...,...,...,...
1225,XSV8808,1,0,0
1226,XSV8811,1,0,0
1227,YI8800LP,6,0,0
1228,YI8802,21,0,0


In [29]:
cycles

,source,start_time,ack_time,ok_time,ack_delay,ok_delay
0,$ACTIVITY_330413F2352AB609,2025-01-01 01:59:46,NaT,NaT,NaN,NaN
1,AAH10023,2025-02-17 05:36:00,2025-02-17 05:37:00,2025-02-17 05:38:00,1.0,2.00
2,AAH10026,2025-03-05 07:12:00,2025-03-05 07:12:00,2025-03-05 07:12:00,0.0,0.00
3,AAH10026,2025-03-05 07:13:00,NaT,2025-03-05 07:13:00,NaN,0.00
4,AAH10027,2025-01-22 00:10:12,NaT,2025-01-22 00:10:15,NaN,0.05
...,...,...,...,...,...,...
83393,YI8841,2025-03-14 22:44:00,2025-03-14 22:45:00,2025-03-14 22:45:00,1.0,1.00
83394,YI8841,2025-03-14 22:46:00,NaT,2025-03-14 22:47:00,NaN,1.00
83395,YI8841,2025-03-14 22:49:00,NaT,2025-03-14 22:50:00,NaN,1.00
83396,YI8841,2025-03-14 22:55:00,NaT,2025-03-14 22:55:00,NaN,0.00


# Roungh Practices

In [12]:
df = df.copy()
df["Event Time"] = pd.to_datetime(df["Event Time"])
df = df.sort_values(["Source", "Event Time"])

"""Prepare and clean alarm event data"""
df = df.copy()
df['Event Time'] = pd.to_datetime(df['Event Time'], errors='coerce')
df['Action'] = df['Action'].astype(str).str.strip().replace({'nan': ''})
# df = df.dropna(subset=['Event Time', 'Source'])
# df = df.sort_values('Event Time')
df = df.sort_values(["Source", "Event Time"])
# df['Action'] = df['Action'].fillna('').str.upper().str.strip()



In [13]:
unique = df.groupby("Source").apply(count_unique_alarms).reset_index(name="Unique_Alarms")


C:\Users\hmaba\AppData\Local\Temp\ipykernel_22128\3220716368.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique = df.groupby("Source").apply(count_unique_alarms).reset_index(name="Unique_Alarms")


In [14]:
unique

,Source,Unique_Alarms
0,$ACTIVITY_330413F2352AB609,1
1,AAH10023,1
2,AAH10026,2
3,AAH10027,22
4,AAH10028,1
...,...,...
1225,XSV8808,1
1226,XSV8811,1
1227,YI8800LP,6
1228,YI8802,21


In [15]:
analyzed = df.groupby("Source", group_keys=False).apply(detect_stale_chatter)


C:\Users\hmaba\AppData\Local\Temp\ipykernel_22128\1519528932.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  analyzed = df.groupby("Source", group_keys=False).apply(detect_stale_chatter)


In [16]:
analyzed

,Event Time,Location Tag,Source,Condition,Action,Priority,Description,Value,Units,is_stale,is_chattering
1441,2025-01-01 01:59:46,01,$ACTIVITY_330413F2352AB609,Command,,J 00,RCM;$330013f2cdc9e162-0006,Start,NaN,False,False
1524,2025-01-01 01:59:47,01,$ACTIVITY_330413F2352AB609,Formula,,J 00,DATA;$330013f2cdc9e162-0006,NaN,NaN,False,False
1523,2025-01-01 01:59:47,01,$ACTIVITY_330413F2352AB609,Formula,,J 00,DATA;$330013f2cdc9e162-0006,NaN,PPM,False,False
1522,2025-01-01 01:59:47,01,$ACTIVITY_330413F2352AB609,Formula,,J 00,DATA;$330013f2cdc9e162-0006,NaN,MIN,False,False
1521,2025-01-01 01:59:47,01,$ACTIVITY_330413F2352AB609,Formula,,J 00,DATA;$330013f2cdc9e162-0006,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...
869728,2025-03-14 22:55:00,8800,YI8841,OFFNRM,,L 00,PURGE DEMAND,PURGE,NaN,False,False
869731,2025-03-14 22:55:00,8800,YI8841,OFFNRM,OK,L 00,PURGE DEMAND,NORMAL,NaN,False,True
869750,2025-03-14 22:56:00,8800,YI8841,OFFNRM,,L 00,PURGE DEMAND,PURGE,NaN,False,False
869755,2025-03-14 22:57:00,8800,YI8841,OFFNRM,OK,L 00,PURGE DEMAND,NORMAL,NaN,False,False


In [17]:
stale = analyzed.groupby("Source")["is_stale"].sum().reset_index(name="Stale_Count")

In [18]:
chat = analyzed.groupby("Source")["is_chattering"].sum().reset_index(name="Chattering_Count")

cycles = get_alarm_cycles(df)

In [21]:
if cycles.empty:
        avg_ack = avg_ok = completion = 0
else:
    avg_ack = cycles["ack_delay"].mean()
    avg_ok = cycles["ok_delay"].mean()
    completion = cycles["ok_time"].notnull().mean() * 100

df["Date"] = df["Event Time"].dt.date
per_day = df.groupby("Date").size()
hrs = (df["Event Time"].max() - df["Event Time"].min()).total_seconds() / 3600
avg_day = per_day.mean()
avg_hr = len(df) / hrs if hrs else 0
avg_10m = avg_hr / 6
overlimit_pct = (per_day > 288).mean() * 100

summary = unique.merge(stale, on="Source", how="outer").merge(chat, on="Source", how="outer").fillna(0)
kpis = dict(
    avg_ack_delay_min=avg_ack,
    avg_ok_delay_min=avg_ok,
    completion_rate_pct=completion,
    avg_alarms_per_day=avg_day,
    avg_alarms_per_hour=avg_hr,
    avg_alarms_per_10min=avg_10m,
    days_over_288_alarms_pct=overlimit_pct,
)

In [22]:
summary

,Source,Unique_Alarms,Stale_Count,Chattering_Count
0,$ACTIVITY_330413F2352AB609,1,0,44375
1,AAH10023,1,0,0
2,AAH10026,2,0,0
3,AAH10027,22,0,2
4,AAH10028,1,0,0
...,...,...,...,...
1225,XSV8808,1,0,0
1226,XSV8811,1,0,0
1227,YI8800LP,6,0,0
1228,YI8802,21,0,0


In [23]:
kpis

{'avg_ack_delay_min': np.float64(140.80233469349903),
 'avg_ok_delay_min': np.float64(185.6824968371187),
 'completion_rate_pct': np.float64(98.88366627497062),
 'avg_alarms_per_day': np.float64(12044.72),
 'avg_alarms_per_hour': 498.833382417906,
 'avg_alarms_per_10min': 83.13889706965101,
 'days_over_288_alarms_pct': np.float64(100.0)}

In [24]:
cycles

,source,start_time,ack_time,ok_time,ack_delay,ok_delay
0,$ACTIVITY_330413F2352AB609,2025-01-01 01:59:46,NaT,NaT,NaN,NaN
1,AAH10023,2025-02-17 05:36:00,2025-02-17 05:37:00,2025-02-17 05:38:00,1.0,2.00
2,AAH10026,2025-03-05 07:12:00,2025-03-05 07:12:00,2025-03-05 07:12:00,0.0,0.00
3,AAH10026,2025-03-05 07:13:00,NaT,2025-03-05 07:13:00,NaN,0.00
4,AAH10027,2025-01-22 00:10:12,NaT,2025-01-22 00:10:15,NaN,0.05
...,...,...,...,...,...,...
83393,YI8841,2025-03-14 22:44:00,2025-03-14 22:45:00,2025-03-14 22:45:00,1.0,1.00
83394,YI8841,2025-03-14 22:46:00,NaT,2025-03-14 22:47:00,NaN,1.00
83395,YI8841,2025-03-14 22:49:00,NaT,2025-03-14 22:50:00,NaN,1.00
83396,YI8841,2025-03-14 22:55:00,NaT,2025-03-14 22:55:00,NaN,0.00
